In [ ]:
pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken

In [ ]:
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

# set seed
seed = 24
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
# define data path
DATA_PATH = './obesity_data/'
AOAI_PATH = './aoai/'

alldocs_df = pd.read_pickle(DATA_PATH + '/alldocs_df.pkl')

API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 



In [ ]:
openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01" 

r = requests.get(url, headers={"api-key": API_KEY})

#print(r.text)

In [ ]:
#Very minimal cleansing as discussed in the AOAI tutorial
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

alldocs_df['text_clean']= alldocs_df["text"].apply(lambda x : normalize_text(x))

In [ ]:
#Need to tokenize this for Azure Open AI, don't plan on splitting as they all fit within token limit
tokenizer = tiktoken.get_encoding("cl100k_base")
alldocs_df['n_tokens'] = alldocs_df["text_clean"].apply(lambda x: len(tokenizer.encode(x)))

print('# too big:',len(alldocs_df[alldocs_df.n_tokens>=8192]))
print('Total Number of Tokens:',sum(alldocs_df['n_tokens']))   


In [ ]:
#This retrieves the embedding.  Since there is a cost, this is commented out.  To use this, you need to set two environment variables
#alldocs_df['ada_v2'] = alldocs_df["text_clean"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) 
# engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
#alldocs_df.to_pickle(AOAI_PATH + '/alldocs_df_aoai.pkl') 

#commented embdding call because there is a charge - loading from pkl file to do analysis
alldocs_df_aoai = pd.read_pickle(AOAI_PATH + '/alldocs_df_aoai.pkl') 


In [ ]:
alldocs_df_aoai.head(40)